In [11]:
import json
import os

import json
with open('/content/Eda_Boosting_networkanalytics_NLP_errorfunction_Ordinaryregression.json') as json_file:
    data = json.load(json_file)

In [12]:
data = data['data'][0]
data=data['paragraphs']

In [18]:
for i in data:
  d=i['qas']

questions=[]
answers=[]

for j in d:
  questions.append(j['question'])
  for k in j['answers']:
    answers.append(k['text'])
print('Done')

Done


In [23]:
questions[:5]

['What is Heat map',
 'What is EDA library',
 'How can you define an empirical relationship in dataset',
 'Which plots used for Plotting the raw data',
 'What is the purpose of visualization']

In [22]:
answers[:5]

['Heat map is a graphical representation of data where values are depicted by color',
 '.Pandas Profiling is the auto EDA library is an open source option that is written in python',
 'Mean, median, and mode are closely connected by the following relations called an empirical relationship',
 'For row data histograms, bihistograms, probability plots, lag plots, block plots, and Youden plots',
 '.It gives assessment not exactness']

In [35]:
import pandas as pd
df=pd.DataFrame({'questions':questions,'answers':answers})
df.head()

,questions,answers
0,What is Heat map,Heat map is a graphical representation of data...
1,What is EDA library,.Pandas Profiling is the auto EDA library is a...
2,How can you define an empirical relationship i...,"Mean, median, and mode are closely connected b..."
3,Which plots used for Plotting the raw data,"For row data histograms, bihistograms, probabi..."
4,What is the purpose of visualization,.It gives assessment not exactness


In [25]:
!pip install autocorrect
import nltk
import re
from autocorrect import Speller
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 622 kB 5.0 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=951ab8b35a1fca1b79b8714b1902693cbbe116a4447d2870c2ef6412045e6ad2
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [26]:
len(df)

1127

In [28]:
for idx , row in df.iterrows():
  print(row['questions'])
  print(row['answers'])
  break

What is Heat map
Heat map is a graphical representation of data where values are depicted by color


In [31]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
from tqdm import tqdm

def preprocess(df):
  spell = Speller(lang= 'en')
  lemmatizer = WordNetLemmatizer()
  stop_words = stopwords.words('english')

  for idx , row in tqdm(df.iterrows()):
    row['questions']=row['questions'].lower()
    row['answers']=row['answers'].lower()
    re.sub(r'([^\s\w]|_)+', ' ', row['questions'])
    re.sub(r'([^\s\w]|_)+', ' ', row['answers'])

    row['questions'] = spell(row['questions'])
    row['answers'] = spell(row['answers'])

    row['questions'] = word_tokenize(row['questions'])
  
    row['questions'] = ' '.join([j for j in row['questions'] if j not in stop_words])
  

In [37]:
preprocess(df)

1127it [02:04,  9.04it/s]


In [38]:
df.head()

,questions,answers
0,heat map,heat map is a graphical representation of data...
1,eda library,.panda profiling is the auto eda library is an...
2,define empirical relationship dataset,"mean, median, and mode are closely connected b..."
3,plots used plotting raw data,"for row data histograms, bihistograms, probabi..."
4,purpose visualization,.it gives assessment not exactness


In [39]:
import random
import string 

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [72]:
def preprocess_text(text):
  spell = Speller(lang= 'en')
  lemmatizer = WordNetLemmatizer()
  stop_words = stopwords.words('english')

  text=text.lower()
  re.sub(r'([^\s\w]|_)+', ' ', text)
  text=spell(text)
  text=word_tokenize(text)
  text=' '.join([j for j in text if j not in stop_words])
  return text

In [92]:
def response(user_response,question_token , answer_token):
    robo_response=''
    TfidfVec = TfidfVectorizer( stop_words='english')
    tfidf = TfidfVec.fit_transform(question_token)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+answer_token[idx]
        return robo_response


In [94]:
flag=True
print("JarVis: My name is JarVis. I will answer your queries about Data Science. If you want to exit, type Bye!")
question_token = df['questions'].values.tolist()
answer_token  = df['answers'].values.tolist()

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("JarVis: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("JarVis: "+greeting(user_response))
            else:
                user_response=preprocess_text(user_response)
                question_token.append(user_response)
                #word_tokens=word_tokens+nltk.word_tokenize(user_response)
                #final_words=list(set(word_tokens))
                print("JarVis: ",end="")
                print(response(user_response,question_token,answer_token ))
                question_token.remove(user_response)
    else:
        flag=False
        print("JarVis: Bye! take care..")  

JarVis: My name is JarVis. I will answer your queries about Data Science. If you want to exit, type Bye!
heat map
JarVis: heat map is a graphical representation of data where values are depicted by color
bye
JarVis: Bye! take care..
